# Test - Check Retention Score Dates
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Apr 30, 2022

In [1]:
import awswrangler as wr
import boto3
import pandas as pd

In [4]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

SVWC-BKNM
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [10]:
session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
client = boto3.client('redshift')

### Let's create a list of team db's that we'd like to check for fresh scores:

In [14]:
CLUSTER = "qa-app"
SCHEMA = "dw"
TABLE = "customerretentionscores"
USER = "admin"

TEAMS = [
    {"cluster": "qa-app","dbname": "stlrahl"},
    {"cluster": "qa-app","dbname": "stlrcfl"},
    {"cluster": "qa-app","dbname": "stlrmilb"},
    {"cluster": "qa-app-elbu","dbname": "stlrmilbseadogs"},
    {"cluster": "qa-app-elbu","dbname": "stlrmilbwindsurge"},
    {"cluster": "qa-app","dbname": "stlrrays"},
    {"cluster": "qa-app","dbname": "stlrmls"},
    {"cluster": "qa-app","dbname": "stlrtrailblazers"},
    {"cluster": "qa-app","dbname": "stlrcanucks"},
    {"cluster": "qa-app","dbname": "stlrcoyotes"},
    {"cluster": "qa-app","dbname": "stlrflames"},
    {"cluster": "qa-app","dbname": "stlrnhlpanthers"},
    {"cluster": "qa-app","dbname": "stlrvgk"},
    {"cluster": "qa-app","dbname": "stlrnll"},
    {"cluster": "qa-app-elbu","dbname": "stlrechleverblades"},
    {"cluster": "qa-app","dbname": "stlrwhl"}
]

### Now we can loop through each datatbase, connect to it and check the retention scores:

In [15]:
print(f"RETENTION SCORE REPORT FOR QA:")

results = []
for team in TEAMS:

    conn = wr.data_api.redshift.connect(
        cluster_id = team['cluster'],
        database =  team['dbname'].lower(),
        db_user = USER
    )
    
    print(f" > Checking scores in: {team['dbname']}.{SCHEMA}.{TABLE}")

    sql = f"""
        SELECT count(*), lkupclientid, MAX(scoredate) as max_score_date
        FROM {team['dbname'].lower()}.{SCHEMA}.{TABLE} 
        GROUP BY lkupclientid
    """

    df = wr.data_api.redshift.read_sql_query(
        sql = sql, 
        con = conn
    )

    df["dbname"] = team['dbname']
    results.append(df)

df_results = pd.concat(results)

df_results.shape


RETENTION SCORE REPORT FOR QA:
 > Checking scores in: stlrahl.dw.customerretentionscores


ValidationException: An error occurred (ValidationException) when calling the ExecuteStatement operation: Redshift endpoint doesn't exist in this region.

In [11]:
df_results = df_results.sort_values('max_score_date')
df_results

,count,lkupclientid,max_score_date,dbname
15,3984,51,2022-04-27,stlrmilb
1,255027,31,2022-04-28,stlrmls
0,281997,45,2022-04-28,stlrrays
0,169261,56,2022-05-04,stlrahl
0,120639,33,2022-05-04,stlrvgk
0,111592,93,2022-05-04,stlrnhlpanthers
0,1919370,36,2022-05-04,stlrflames
0,1082033,55,2022-05-04,stlrcoyotes
0,3622806,7,2022-05-04,stlrcanucks
2,93950,13,2022-05-04,stlrmls


### Done